In [ ]:
import os
import SimpleITK as sitk
import numpy as np

# Function to reduce voxel size using erosion and resampling
def reduce_voxel_size(input_image_path, output_image_path, target_voxel_size_mm=(5.0, 5.0, 5.0)):
    # Load the input MRI image
    input_image = sitk.ReadImage(input_image_path)

    # Convert the input image to a supported pixel type (e.g., sitkInt16)
    input_image = sitk.Cast(input_image, sitk.sitkInt16)

    # Compute the target voxel spacing
    target_spacing = np.array(target_voxel_size_mm)

    # Get the current voxel spacing of the input image
    current_spacing = input_image.GetSpacing()

    # Compute the resampling factor based on the ratio of target spacing to current spacing
    resampling_factor = current_spacing / target_spacing

    # Compute the target size (dimensions) for resampling based on resampling factor
    target_size = np.ceil(np.array(input_image.GetSize()) * resampling_factor).astype(int).tolist()

    # Create the identity transformation for resampling
    transform = sitk.Transform()

    # Resample the input image to achieve the target voxel size and dimensions
    resampled_image = sitk.Resample(input_image, target_size, transform,
                                     sitk.sitkLinear, input_image.GetOrigin(),
                                     target_spacing, input_image.GetDirection(),
                                     0, input_image.GetPixelID())

    # Save the resampled image to the output path
    sitk.WriteImage(resampled_image, output_image_path)

    print(f"Image successfully resampled to {target_voxel_size_mm} mm voxel size and saved to {output_image_path}")

# Input and output folders
input_folder = "/Intern Programming/data/UMCGOPC_MRI_nii_zscoreinHNmask"
output_folder = '/Intern Programming/data/output/UMCGOPC_MRI_nii_zscoreinHNmask_5mm'

# Desired voxel size in mm
target_voxel_size_mm = (5.0, 5.0, 5.0)

# Iterate through all NIfTI files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.nii.gz'):
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, filename)

        # Reduce voxel size for the current input image and save to output folder
        reduce_voxel_size(input_image_path, output_image_path, target_voxel_size_mm)


In [9]:
import SimpleITK as sitk

# Function to get voxel size (spacing) from a NIfTI file
def get_voxel_size(nii_file_path):
    # Load the NIfTI image
    image = sitk.ReadImage(nii_file_path)

    # Get the voxel spacing (size of each voxel in millimeters)
    voxel_size = image.GetSpacing()

    return voxel_size

# Example usage:
nii_file_path = "/Intern Programming/data/output/UMCGOPC_MRI_nii_zscoreinHNmask_5mm/UMCG-0020715_t1.nii.gz"

# Get the voxel size (spacing) of the NIfTI file
voxel_size = get_voxel_size(nii_file_path)

# Print the voxel size (spacing) in millimeters
print(f"Voxel size (spacing): {voxel_size} mm")


Voxel size (spacing): (5.0, 5.0, 5.0) mm


In [ ]:
# The whole original one 

import os
import SimpleITK as sitk

import pandas as pd

# Define the path to the directory containing the CT images and GTV masks
path_to_data_t1 = "/Intern Programming/data/output/UMCGOPC_MRI_nii_zscoreinHNmask"
path_to_data_t1gt0tt = "/Intern Programming/data/t1t2_gtv"
# clinical data path 
clc_data_path  = ('/Intern Programming/data/OPCdigits_333_split_t1good.csv') 
clc_data = pd.read_csv( clc_data_path ).drop(columns=['Unnamed: 0'])


# Get a list of the CT images and GTV masks in the directory
patients_number = 333
files = clc_data[ 'PatientID' ]
IDs = range(patients_number)

# Load each CT image and GTV mask into a list
mri_images = [os.path.join(path_to_data_t1, x + '_t1.nii.gz') for x in files]
gtv_masks = [os.path.join(path_to_data_t1gt0tt, x + '_gtvpttot1.nii.gz') for x in files] # gtvpt or gtvtot


from radiomics import featureextractor

# Create a PyRadiomics feature extractor object
extractor = featureextractor.RadiomicsFeatureExtractor()

# Specify the features you want to extract
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')

# Specify the discretization of the image intensities
extractor.settings["binWidth"] = 25 # look at papers
extractor.settings["resampledPixelSpacing"] = (1.0, 1.0, 1.0) 
 
# Specify the resampling of the image and mask
extractor.settings["interpolator"] = "sitkLinear"
extractor.settings["resamplingInterpolator"] = "sitkNearestNeighbor"
extractor.settings["correctMask"] = True


# Initialize an empty list to store DataFrames for each patient
feature_dfs = []

# Iterate over patients and extract features
for i, (patient_id, ct_image_path, gtv_mask_path) in enumerate(zip(files, mri_images, gtv_masks)):
    #try:
        ct_image = sitk.ReadImage(ct_image_path)
        gtv_mask = sitk.ReadImage(gtv_mask_path)
        features = extractor.execute(ct_image, gtv_mask)

        
        
        # Add patient ID to features dictionary
        features['PatientID'] = patient_id
        
        # Convert features dictionary to DataFrame and append to feature_dfs list
        #features_df = pd.DataFrame(features, index=[0])  # Create DataFrame with a single row
        feature_dfs.append(features)
   




In [22]:

import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor

# Define paths to input directories
path_to_data_t2 = "/Intern Programming/data/UMCGOPC_MRI_nii_zscoreinHNmask"
path_to_data_t2gt0tt = "/Intern Programming/data/t1t2_gtv"
clc_data_path = '/Intern Programming/data/OPCdigits_268_split_t2good.csv'

# Load clinical data
clc_data = pd.read_csv(clc_data_path).drop(columns=['Unnamed: 0'])

# Get list of patient IDs and corresponding files
files = clc_data['PatientID']

# Define lists of CT images and GTV masks
mri_images = [os.path.join(path_to_data_t2, f"{x}_t2.nii.gz") for x in files]
gtv_masks = [os.path.join(path_to_data_t2gt0tt, f"{x}_gtvpttot2.nii.gz") for x in files]

# Create a PyRadiomics feature extractor object
extractor = featureextractor.RadiomicsFeatureExtractor()

# Specify the features you want to extract
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('gldm')
extractor.enableFeatureClassByName('glrlm')
extractor.enableFeatureClassByName('glszm')
extractor.enableFeatureClassByName('ngtdm')

# Specify the discretization of the image intensities
extractor.settings["binWidth"] = 0.1  # Intensity discretization bin width
extractor.settings["resampledPixelSpacing"] = (0.5, 0.5, 1.0)  # Resampled voxel spacing

# Specify the resampling of the image and mask
extractor.settings["interpolator"] = "sitkLinear"
extractor.settings["resamplingInterpolator"] = "sitkNearestNeighbor"
extractor.settings["correctMask"] = True  # Correct mask if it extends outside the image bounds

# Initialize an empty list to store DataFrames for each patient
feature_dfs = []

# Iterate over patients and extract features
for patient_id, ct_image_path, gtv_mask_path in zip(files, mri_images, gtv_masks):
    try:
        # Load CT image and GTV mask
        ct_image = sitk.ReadImage(ct_image_path)
        gtv_mask = sitk.ReadImage(gtv_mask_path)
        
        # Extract radiomics features using the feature extractor
        features = extractor.execute(ct_image, gtv_mask)
        
        # Add patient ID to features dictionary
        features['PatientID'] = patient_id
        
        # Convert features OrderedDict to DataFrame
        features_df = pd.DataFrame.from_dict(features, orient='index').T
        
        # Append features DataFrame to list of DataFrames
        feature_dfs.append(features_df)
        
        print(f"Successfully extracted features for patient {patient_id}")
    except Exception as e:
        print(f"Error extracting features for patient {patient_id}: {e}")

# Check if any features were successfully extracted
if feature_dfs:
    # Concatenate all feature DataFrames into a single DataFrame
    all_features = pd.concat(feature_dfs, ignore_index=True)
    
    # Print the DataFrame containing all extracted features
    print(all_features)
else:
    print("No features were extracted successfully.")



GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-0573601


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-6395952


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


...

Successfully extracted features for patient UMCG-0412861


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-5325319


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Successfully extracted features for patient UMCG-2291679
    diagnostics_Versions_PyRadiomics diagnostics_Versions_Numpy  \
0                             v3.1.0                     1.26.4   
1                             v3.1.0                     1.26.4   
2                             v3.1.0                     1.26.4   
3                             v3.1.0                     1.26.4   
4                             v3.1.0                     1.26.4   
..                               ...                        ...   
263                           v3.1.0                     1.26.4   
264                           v3.1.0                     1.26.4   
265                           v3.1.0                     1.26.4   
266                           v3.1.0                     1.26.4   
267                           v3.1.0                     1.26.4   

    diagnostics_Versions_SimpleITK diagnostics_Versions_PyWavelet  \
0                            2.3.1                          1.5.0   


In [23]:
df = pd.DataFrame(all_features)
df.to_csv(r'/Intern Programming/data/output/' + 'umcg_radiomicsfeature_t2_GTVPTtot.csv' )

In [6]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_erosion

# Function to perform erosion on an MRI scan and save the result
def perform_erosion(input_path, output_path, structuring_element):
    # Load MRI scan data using nibabel
    mri_scan = nib.load(input_path)
    mri_data = mri_scan.get_fdata()

    # Perform erosion on the MRI data
    eroded_data = binary_erosion(mri_data, structure=structuring_element)

    # Create a new NIfTI image from the eroded data
    eroded_mri_scan = nib.Nifti1Image(eroded_data, affine=mri_scan.affine, header=mri_scan.header)

    # Save the eroded MRI scan to the specified output path
    nib.save(eroded_mri_scan, output_path)
    print(f"Erosion completed and saved to: {output_path}")

input_folder = "/Intern Programming/data/t1t2_gtv"
output_folder = '/Intern Programming/data/output/t1t2gtv_1mm'

# Define the structuring element for erosion (e.g., 3x3x3 cube)
structuring_element = np.ones((2, 2, 2)) 

# Iterate over each MRI scan file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.nii.gz') or filename.endswith('.nii'):
        # Construct full paths for input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'{filename}')  # Modify output filename if needed

        # Perform erosion on the MRI scan and save the result
        perform_erosion(input_path, output_path, structuring_element)





Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0020715_gtvpttot1.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0020715_gtvpttot2.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0020715_gtvtottot1.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0020715_gtvtottot2.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0021879_gtvpttot1.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0021879_gtvpttot2.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0021879_gtvtottot1.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0021879_gtvtottot2.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_1mm\UMCG-0059896_gtvpttot1.nii.gz
Erosion completed and saved to: /Intern Pr

In [1]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_erosion

# Function to perform erosion on an MRI scan and save the result
def perform_erosion(input_path, output_path, structuring_element):
    # Load MRI scan data using nibabel
    mri_scan = nib.load(input_path)
    mri_data = mri_scan.get_fdata()

    # Perform erosion on the MRI data
    eroded_data = binary_erosion(mri_data, structure=structuring_element)

    # Create a new NIfTI image from the eroded data
    eroded_mri_scan = nib.Nifti1Image(eroded_data, affine=mri_scan.affine, header=mri_scan.header)

    # Save the eroded MRI scan to the specified output path
    nib.save(eroded_mri_scan, output_path)
    print(f"Erosion completed and saved to: {output_path}")

input_folder = "/Intern Programming/data/t1t2_gtv"
output_folder = '/Intern Programming/data/output/t1t2gtv_5mm'

# Define the structuring element for erosion (e.g., 3x3x3 cube)
structuring_element = np.ones((5, 5, 5))

# Iterate over each MRI scan file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.nii.gz') or filename.endswith('.nii'):
        # Construct full paths for input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'{filename}')  # Modify output filename if needed

        # Perform erosion on the MRI scan and save the result
        perform_erosion(input_path, output_path, structuring_element)


Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0020715_gtvpttot1r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0020715_gtvpttot2r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0020715_gtvtottot1r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0020715_gtvtottot2r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0021879_gtvpttot1r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0021879_gtvpttot2r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0021879_gtvtottot1r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0021879_gtvtottot2r.nii.gz
Erosion completed and saved to: /Intern Programming/data/output/t1t2gtv_5mm\UMCG-0059896_gtvpttot1r.nii.gz
Erosion completed and saved to: /

In [ ]:
df = pd.DataFrame(all_features)
df.head()

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 25 14:32:24 2024

@author: MaB
"""
import numpy as np
import SimpleITK as sitk
import os

def resample_image(itk_image, out_spacing=[1.0, 1.0, 1.0], is_label=False):
    original_spacing = itk_image.GetSpacing()
    
    original_size = itk_image.GetSize()
    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
    ]
  
    '''
    # keep the same size with Spacing_function in MONAI.  
    out_size = [
        int(original_size[0] * (original_spacing[0] / out_spacing[0])),
        int(original_size[1] * (original_spacing[1] / out_spacing[1])),
        int(original_size[2] * (original_spacing[2] / out_spacing[2]))
    ]
    '''
    
    print ( out_size )

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    #resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
    
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
        resample.SetDefaultPixelValue( 0 )
    else:
        resample.SetInterpolator(sitk.sitkLinear)
        resample.SetDefaultPixelValue( np.min( sitk.GetArrayFromImage(itk_image)) )

    return resample.Execute(itk_image) 



# for all patients together for t1 and t2
# load path
path_to_sitk_image = "/Intern Programming/data/UMCGOPC_MRI_nii_zscoreinHNmask"
# save path
path_to_save  = r'E:\Intern Programming\data\output\UMCGOPC_MRI_nii_zscoreinHNmask_1mm'
image_names = os.listdir( path_to_sitk_image )

image_names =  sorted( image_names )

for image_name in image_names:
    image_name_long = os.path.join( path_to_sitk_image, image_name )
    image =  sitk.ReadImage( image_name_long )
    newimage  = resample_image( image, out_spacing = [1.0, 1.0, 1.0], is_label = False )
    sitk.WriteImage( newimage, os.path.join( path_to_save, image_name[ 0 : 15] + 'r.nii.gz'  ) )   



# for all patients together for t1_gtv and t2_gtv
# load path
path_to_sitk_image = "/Intern Programming/data/t1t2_gtv"
path_to_save  = r'E:\Intern Programming\data\output\t1t2gtv_1mm'
image_names = os.listdir( path_to_sitk_image )   

image_names =  sorted( image_names )   
#image_names = [ 'UMCG-2116597_gtvpttot1.nii.gz' ] 

for image_name in image_names:   
    
    image_name_long = os.path.join( path_to_sitk_image, image_name )   
    image =  sitk.ReadImage( image_name_long )   
    newimage  = resample_image( image, out_spacing = [1.0, 1.0, 1.0], is_label = True )   
    
    name_part, extension_part = image_name.split('.', 1)   
    newimage[ newimage > 0 ] = 1   
    
    print ( sitk.GetArrayFromImage(newimage).shape )   
    sitk.WriteImage( newimage, os.path.join( path_to_save, name_part + 'r.nii.gz'  ) )   

    

[200, 200, 155]
[200, 200, 129]
[200, 200, 126]
[200, 200, 126]
[200, 200, 155]
[200, 200, 129]
[200, 200, 130]
[200, 200, 126]
[200, 200, 155]
[200, 200, 126]
[180, 180, 126]
[200, 200, 126]
[200, 200, 176]
[200, 200, 129]
[180, 180, 126]
[200, 200, 126]
[180, 180, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 155]
[200, 200, 135]
[180, 180, 151]
[220, 220, 144]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 140]
[200, 200, 155]
[200, 200, 129]
[200, 200, 155]
[200, 200, 129]
[180, 180, 126]
[200, 200, 126]
[200, 200, 155]
[200, 200, 142]
[200, 200, 155]
[200, 200, 129]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[200, 200, 169]
[200, 200, 126]
[200, 200, 126]
[200, 200, 126]
[180, 180, 140]
[200, 200, 126]
[180, 180, 140]
[200, 200, 133]
[200, 200, 126]
[208, 230, 140]
[208, 230, 140]
[200, 200, 140]
[200, 200, 126]
[180, 180, 126]
[200, 200, 126]
[180, 180, 126]
[200, 200, 126]
[200, 20